# deep-inference E2E User Test (Poisson)

Simulates a new user installing and testing `deep-inference` from PyPI.

Tests **4 different DGPs** to demonstrate generalization:
1. **Simple Linear**: Basic heterogeneity
2. **Strong Heterogeneity**: Larger coefficients
3. **Nonlinear**: sin/quadratic functions
4. **Multi-dimensional**: 5D covariates

**Model**: Y ~ Poisson(λ), where λ = exp(α(X) + β(X)·T)

## 1. Installation & Setup

In [1]:
# Install from PyPI
!pip install 'deep-inference>=0.1.1' --quiet

import deep_inference
print("deep-inference installed successfully!")

deep-inference installed successfully!


In [2]:
import numpy as np
import warnings
import statsmodels.api as sm
from deep_inference import structural_dml

warnings.filterwarnings('ignore')
print("All imports successful!")

All imports successful!


## 2. Define DGPs

In [3]:
# Common config
N = 1000
M_ORACLE = 50  # Oracle MC replications (reduced for speed)
EPOCHS = 100
N_FOLDS = 50
HIDDEN_DIMS = [64, 32]
LR = 0.01

# DGP definitions (Poisson: Y ~ Poisson(exp(alpha + beta*T)))
DGPS = {
    'Simple Linear': {
        'desc': 'alpha=1.0+0.2X, beta=0.3+0.1X',
        'dim': 1,
        'alpha_fn': lambda X: 1.0 + 0.2*X[:, 0],
        'beta_fn': lambda X: 0.3 + 0.1*X[:, 0],
        'mu_true': 0.3,
    },
    'Strong Hetero': {
        'desc': 'alpha=0.5+0.3X, beta=0.3+0.3X',
        'dim': 1,
        'alpha_fn': lambda X: 0.5 + 0.3*X[:, 0],
        'beta_fn': lambda X: 0.3 + 0.3*X[:, 0],
        'mu_true': 0.3,
    },
    'Nonlinear': {
        'desc': 'alpha=1.0+0.2sin(piX), beta=0.3+0.1X^2',
        'dim': 1,
        'alpha_fn': lambda X: 1.0 + 0.2*np.sin(np.pi*X[:, 0]),
        'beta_fn': lambda X: 0.3 + 0.1*X[:, 0]**2,
        'mu_true': 0.3 + 0.1*1.0,  # E[X^2] = Var(X) + E[X]^2 = 1
    },
    'Multi-dim (5D)': {
        'desc': 'alpha=1.0+0.15X1+0.1X2, beta=0.3+0.2X1',
        'dim': 5,
        'alpha_fn': lambda X: 1.0 + 0.15*X[:, 0] + 0.1*X[:, 1],
        'beta_fn': lambda X: 0.3 + 0.2*X[:, 0],
        'mu_true': 0.3,
    },
}

print("DGPs defined:")
for name, dgp in DGPS.items():
    print(f"  {name}: {dgp['desc']}, mu*={dgp['mu_true']}")

DGPs defined:
  Simple Linear: alpha=1.0+0.2X, beta=0.3+0.1X, mu*=0.3
  Strong Hetero: alpha=0.5+0.3X, beta=0.3+0.3X, mu*=0.3
  Nonlinear: alpha=1.0+0.2sin(piX), beta=0.3+0.1X^2, mu*=0.4
  Multi-dim (5D): alpha=1.0+0.15X1+0.1X2, beta=0.3+0.2X1, mu*=0.3


In [4]:
def generate_data(dgp, n, seed):
    """Generate data from a Poisson DGP."""
    np.random.seed(seed)
    X = np.random.normal(0, 1, (n, dgp['dim']))
    T = np.random.normal(0, 1, n)
    alpha = dgp['alpha_fn'](X)
    beta = dgp['beta_fn'](X)
    lam = np.exp(alpha + beta*T)
    Y = np.random.poisson(lam).astype(float)
    return X, T, Y, alpha, beta

def run_oracle(X, T, Y, mu_true):
    """Run oracle Poisson GLM (1D only)."""
    n = len(Y)
    if X.shape[1] == 1:
        X_flat = X[:, 0]
        X_bar = X_flat.mean()
        X_design = np.column_stack([np.ones(n), X_flat, T, X_flat*T])
        model = sm.GLM(Y, X_design, family=sm.families.Poisson()).fit()
        b0, b1 = model.params[2], model.params[3]
        cov = model.cov_params()
        mu = b0 + b1*X_bar
        var = cov[2,2] + X_bar**2*cov[3,3] + 2*X_bar*cov[2,3] + b1**2*(X_flat.var(ddof=1)/n)
        se = np.sqrt(max(var, 1e-10))
        covers = (mu - 1.96*se) <= mu_true <= (mu + 1.96*se)
        return {'mu': mu, 'se': se, 'covers': covers}
    else:
        return None  # Oracle not available for multi-dim

def run_nn(X, T, Y, mu_true):
    """Run neural network with IF correction."""
    nn = structural_dml(
        Y=Y, T=T, X=X,
        family='poisson',
        lambda_method='aggregate',  # CRITICAL for Poisson
        epochs=EPOCHS, n_folds=N_FOLDS,
        hidden_dims=HIDDEN_DIMS, lr=LR,
        verbose=False
    )
    beta_hat = nn.theta_hat[:, 1]
    mu_naive = beta_hat.mean()
    se_naive = beta_hat.std() / np.sqrt(len(Y))
    covers_naive = (mu_naive - 1.96*se_naive) <= mu_true <= (mu_naive + 1.96*se_naive)
    covers_if = nn.ci_lower <= mu_true <= nn.ci_upper
    return {
        'mu_naive': mu_naive, 'se_naive': se_naive, 'covers_naive': covers_naive,
        'mu_if': nn.mu_hat, 'se_if': nn.se, 'covers_if': covers_if
    }

print("Helper functions defined.")

Helper functions defined.


## 3. Run All DGPs

In [5]:
results = {}

for dgp_name, dgp in DGPS.items():
    print(f"\n{'='*70}")
    print(f"DGP: {dgp_name}")
    print(f"{'='*70}")
    print(f"  {dgp['desc']}")
    print(f"  dim={dgp['dim']}, mu*={dgp['mu_true']}")
    
    # Oracle MC (only for 1D)
    if dgp['dim'] == 1:
        print(f"\n  Running Oracle MC (M={M_ORACLE})...")
        oracle_mus, oracle_covers = [], []
        for seed in range(1, M_ORACLE+1):
            X, T, Y, _, _ = generate_data(dgp, N, seed)
            r = run_oracle(X, T, Y, dgp['mu_true'])
            oracle_mus.append(r['mu'])
            oracle_covers.append(r['covers'])
        oracle_coverage = 100 * np.mean(oracle_covers)
        oracle_se = np.std(oracle_mus)
        print(f"  Oracle: Coverage={oracle_coverage:.0f}%, Emp SE={oracle_se:.4f}")
    else:
        oracle_coverage = None
        oracle_se = None
        print(f"  Oracle: N/A (multi-dim)")
    
    # NN single run
    print(f"  Running NN (seed=42)...")
    X, T, Y, alpha, beta = generate_data(dgp, N, seed=42)
    nn_result = run_nn(X, T, Y, dgp['mu_true'])
    
    print(f"  NN Naive: mu={nn_result['mu_naive']:.4f}, SE={nn_result['se_naive']:.4f}, Covers={nn_result['covers_naive']}")
    print(f"  NN IF:    mu={nn_result['mu_if']:.4f}, SE={nn_result['se_if']:.4f}, Covers={nn_result['covers_if']}")
    
    results[dgp_name] = {
        'mu_true': dgp['mu_true'],
        'oracle_coverage': oracle_coverage,
        'oracle_se': oracle_se,
        **nn_result
    }


DGP: Simple Linear
  alpha=1.0+0.2X, beta=0.3+0.1X
  dim=1, mu*=0.3

  Running Oracle MC (M=50)...


  Oracle: Coverage=100%, Emp SE=0.0169
  Running NN (seed=42)...


  NN Naive: mu=0.2809, SE=0.0041, Covers=False
  NN IF:    mu=0.2962, SE=0.0208, Covers=True

DGP: Strong Hetero
  alpha=0.5+0.3X, beta=0.3+0.3X
  dim=1, mu*=0.3

  Running Oracle MC (M=50)...


  Oracle: Coverage=98%, Emp SE=0.0226
  Running NN (seed=42)...


  NN Naive: mu=0.2747, SE=0.0095, Covers=False
  NN IF:    mu=0.2382, SE=0.0601, Covers=True

DGP: Nonlinear
  alpha=1.0+0.2sin(piX), beta=0.3+0.1X^2
  dim=1, mu*=0.4

  Running Oracle MC (M=50)...


  Oracle: Coverage=78%, Emp SE=0.0290
  Running NN (seed=42)...


  NN Naive: mu=0.4057, SE=0.0039, Covers=True
  NN IF:    mu=0.4189, SE=0.0269, Covers=True

DGP: Multi-dim (5D)
  alpha=1.0+0.15X1+0.1X2, beta=0.3+0.2X1
  dim=5, mu*=0.3
  Oracle: N/A (multi-dim)
  Running NN (seed=42)...


  NN Naive: mu=0.2864, SE=0.0062, Covers=False
  NN IF:    mu=0.2600, SE=0.0369, Covers=True


## 4. Summary Table

In [6]:
print("="*110)
print("SUMMARY: ALL DGPs (Poisson)")
print("="*110)
print(f"{'DGP':<20} {'mu*':<8} {'Oracle Cov':<12} {'NN Naive':<25} {'NN IF':<25} {'SE Ratio':<10}")
print(f"{'':20} {'':8} {'':12} {'Est / SE / Cov':<25} {'Est / SE / Cov':<25} {'IF/Naive':<10}")
print("-"*110)

for dgp_name, r in results.items():
    oracle_str = f"{r['oracle_coverage']:.0f}%" if r['oracle_coverage'] else "N/A"
    naive_str = f"{r['mu_naive']:.3f} / {r['se_naive']:.4f} / {r['covers_naive']}"
    if_str = f"{r['mu_if']:.3f} / {r['se_if']:.4f} / {r['covers_if']}"
    se_ratio = f"{r['se_if']/r['se_naive']:.1f}x"
    print(f"{dgp_name:<20} {r['mu_true']:<8.2f} {oracle_str:<12} {naive_str:<25} {if_str:<25} {se_ratio:<10}")

print("-"*110)

SUMMARY: ALL DGPs (Poisson)
DGP                  mu*      Oracle Cov   NN Naive                  NN IF                     SE Ratio  
                                           Est / SE / Cov            Est / SE / Cov            IF/Naive  
--------------------------------------------------------------------------------------------------------------
Simple Linear        0.30     100%         0.281 / 0.0041 / False    0.296 / 0.0208 / True     5.1x      
Strong Hetero        0.30     98%          0.275 / 0.0095 / False    0.238 / 0.0601 / True     6.3x      
Nonlinear            0.40     78%          0.406 / 0.0039 / True     0.419 / 0.0269 / True     6.9x      
Multi-dim (5D)       0.30     N/A          0.286 / 0.0062 / False    0.260 / 0.0369 / True     6.0x      
--------------------------------------------------------------------------------------------------------------


In [7]:
print("="*70)
print("KEY FINDINGS (Poisson)")
print("="*70)

# Count coverage
naive_covers = sum(1 for r in results.values() if r['covers_naive'])
if_covers = sum(1 for r in results.values() if r['covers_if'])
total = len(results)

print(f"\nCoverage across {total} DGPs:")
print(f"  NN Naive: {naive_covers}/{total} ({100*naive_covers/total:.0f}%)")
print(f"  NN IF:    {if_covers}/{total} ({100*if_covers/total:.0f}%)")

# SE ratios
se_ratios = [r['se_if']/r['se_naive'] for r in results.values()]
print(f"\nSE Ratio (IF/Naive):")
print(f"  Mean: {np.mean(se_ratios):.1f}x")
print(f"  Range: [{min(se_ratios):.1f}x, {max(se_ratios):.1f}x]")

print(f"\nConclusion:")
print(f"  - Naive SE is {np.mean(se_ratios):.0f}x too small (overconfident)")
print(f"  - IF correction provides valid coverage")
print(f"  - Works across linear, nonlinear, and multi-dimensional DGPs")
print(f"  - CRITICAL: Must use lambda_method='aggregate' for Poisson")

KEY FINDINGS (Poisson)

Coverage across 4 DGPs:
  NN Naive: 1/4 (25%)
  NN IF:    4/4 (100%)

SE Ratio (IF/Naive):
  Mean: 6.1x
  Range: [5.1x, 6.9x]

Conclusion:
  - Naive SE is 6x too small (overconfident)
  - IF correction provides valid coverage
  - Works across linear, nonlinear, and multi-dimensional DGPs
  - CRITICAL: Must use lambda_method='aggregate' for Poisson


## 5. Detailed Results by DGP

In [8]:
for dgp_name, r in results.items():
    print(f"\n{'='*60}")
    print(f"{dgp_name}")
    print(f"{'='*60}")
    print(f"True mu*: {r['mu_true']}")
    if r['oracle_coverage']:
        print(f"Oracle Coverage: {r['oracle_coverage']:.0f}%")
    print(f"\nNN Naive:")
    print(f"  Estimate: {r['mu_naive']:.6f}")
    print(f"  SE: {r['se_naive']:.6f}")
    print(f"  CI: [{r['mu_naive']-1.96*r['se_naive']:.4f}, {r['mu_naive']+1.96*r['se_naive']:.4f}]")
    print(f"  Covers: {r['covers_naive']}")
    print(f"\nNN IF:")
    print(f"  Estimate: {r['mu_if']:.6f}")
    print(f"  SE: {r['se_if']:.6f}")
    print(f"  CI: [{r['mu_if']-1.96*r['se_if']:.4f}, {r['mu_if']+1.96*r['se_if']:.4f}]")
    print(f"  Covers: {r['covers_if']}")
    print(f"\nSE Ratio (IF/Naive): {r['se_if']/r['se_naive']:.1f}x")


Simple Linear
True mu*: 0.3
Oracle Coverage: 100%

NN Naive:
  Estimate: 0.280858
  SE: 0.004073
  CI: [0.2729, 0.2888]
  Covers: False

NN IF:
  Estimate: 0.296180
  SE: 0.020787
  CI: [0.2554, 0.3369]
  Covers: True

SE Ratio (IF/Naive): 5.1x

Strong Hetero
True mu*: 0.3
Oracle Coverage: 98%

NN Naive:
  Estimate: 0.274652
  SE: 0.009484
  CI: [0.2561, 0.2932]
  Covers: False

NN IF:
  Estimate: 0.238172
  SE: 0.060127
  CI: [0.1203, 0.3560]
  Covers: True

SE Ratio (IF/Naive): 6.3x

Nonlinear
True mu*: 0.4
Oracle Coverage: 78%

NN Naive:
  Estimate: 0.405668
  SE: 0.003888
  CI: [0.3980, 0.4133]
  Covers: True

NN IF:
  Estimate: 0.418863
  SE: 0.026947
  CI: [0.3660, 0.4717]
  Covers: True

SE Ratio (IF/Naive): 6.9x

Multi-dim (5D)
True mu*: 0.3

NN Naive:
  Estimate: 0.286420
  SE: 0.006164
  CI: [0.2743, 0.2985]
  Covers: False

NN IF:
  Estimate: 0.259984
  SE: 0.036939
  CI: [0.1876, 0.3324]
  Covers: True

SE Ratio (IF/Naive): 6.0x
